<a href="https://colab.research.google.com/github/sufyanAshraf/AI_Agents/blob/main/pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU \
  pinecone-client==3.1.0 \
  pandas==2.0.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 36.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.0.3 which is incompatible.
plotnine 0.14.5 requires pandas>=2.2.0, but you have pandas 2.0.3 which is incompatible.
xarray 2025.1.2 requires pandas>=2.1, but you have pandas 2.0.3 which is incompatible.
mizani 0.13.1 requires pandas>=2.2.0, but you have pandas 2.0.3 which is incompatible.


In [ ]:
import os
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = 'pcsk_*' # Add Key

# configure client
pc = Pinecone(api_key=api_key, environment ="us-east-1")

In [ ]:
pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 128,
              'host': 'test-m7p6u3a.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'test',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'},
              'tags': None,
              'vector_type': 'dense'},
             {'deletion_protection': 'disabled',
              'dimension': 128,
              'host': 'test2-m7p6u3a.svc.aped-4627-b74a.pinecone.io',
              'metric': 'euclidean',
              'name': 'test2',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'},
              'tags': None,
              'vector_type': 'dense'}]}

In [ ]:
cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'
pc.create_index(name="test", dimension=128, spec=pinecone.ServerlessSpec(cloud=cloud, region=region))

In [ ]:
pc.create_index(name="test2", dimension=128, spec=pinecone.ServerlessSpec(cloud=cloud, region=region), metric="euclidean")

In [ ]:
# wait for index to be ready before connecting
while not pc.describe_index("test").status['ready']:
    time.sleep(1)

In [ ]:
pc.delete_index("test2")

In [ ]:
pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 128,
              'host': 'test-m7p6u3a.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'test',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'},
              'tags': None,
              'vector_type': 'dense'}]}

In [ ]:
import random

In [ ]:
ids = ['a', 'b', 'c', 'd','e']
vec =[[random.random() for _ in range(128)] for v in range(5)]

In [ ]:
index = pc.Index("test")

In [ ]:
index.upsert(vectors=zip(ids, vec))

{'upserted_count': 5}

In [ ]:
import itertools
v_dim =128
v_count =10000


example_data_gen = map(
    lambda i:
        (f'id-{i}', [random.random() for _ in range(v_dim)]),
        range(v_count),
)

def chunks(iterable, batch_size=100):
    """A helper function to break an iterable into chunks of size batch_size."""
    iterator = iter(iterable)
    while True:
        chunk = tuple(itertools.islice(iterator, batch_size))
        if not chunk:
            break
        yield chunk


In [ ]:
for chunk in chunks(example_data_gen, batch_size=100):
    index.upsert(vectors=chunk)

In [ ]:
index.describe_index_stats()

{'dimension': 128,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 10005}},
 'total_vector_count': 10005}

In [ ]:
index.fetch(ids=['id-0', 'id-1'])

#query

In [ ]:
qurey = [[random.random() for _ in range(128)] for _ in range(2)]

In [ ]:
index.query(
    queries=qurey,
    top_k=3,
    include_metadata=True,
    filter={
        "id": {"$eq": "id-0"}
    }
)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Fri, 14 Feb 2025 10:10:58 GMT', 'Content-Type': 'text/plain', 'Content-Length': '88', 'Connection': 'keep-alive', 'server': 'envoy'})
HTTP response body: queries[127]: invalid value 0.79280002065875854 for type type.googleapis.com/QueryVector


In [ ]:
pc.delete_index("test")

In [ ]:
pc.list_indexes()

{'indexes': []}